In [2]:
import tensorflow as tf
import pandas as pd
import platform
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [6]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [7]:
train.head() 

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


In [29]:
test.head(5)

,id,comment_text
0,6044863,==Orphaned non-free media (Image:41cD1jboEvL. ...
1,6102620,::Kentuckiana is colloquial. Even though the ...
2,14563293,"Hello fellow Wikipedians,\nI have just modifie..."
3,21086297,"AKC Suspensions \nThe Morning Call - Feb 24, 2..."
4,22982444,== [WIKI_LINK: Talk:Celts] ==


In [28]:
lens = train.comment_text.str.len()
lens2 =test.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

(395.34186393464859, 595.10207169971216, 5000)

In [10]:
hist = lens.hist(bins=100)

hist.set_xlim(0, 1000)
start, end = hist.get_xlim()
hist.xaxis.set_ticks(np.arange(start, end, 100));

In [11]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
count,9.585100e+04,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000
mean,4.994359e+11,0.096368,0.010068,0.053301,0.003182,0.049713,0.008492,0.897862
std,2.890136e+11,0.295097,0.099832,0.224635,0.056320,0.217352,0.091762,0.302831
min,2.225664e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.473437e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,5.001297e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,7.501088e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999882e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
COMMENT = 'comment_text'
train[COMMENT].fillna('unknown', inplace=True)
test[COMMENT].fillna('unknown', inplace=True)

In [13]:
import re, string
re_token = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [14]:
split = train.comment_text.str.split()
#split.apply(lambda x: [entry.strip(string.punctuation) for entry in x])
#split

In [15]:
vectorizer = TfidfVectorizer()
tokenized_data = vectorizer.fit_transform(train.comment_text)

count_vectorized= CountVectorizer()
count_vectorized_data =count_vectorized.fit_transform(train.comment_text)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
target = train[list_classes].values

In [58]:
#Chechink for shapes macth
target[:int(target.shape[0]*0.6),0].shape, count_vectorized_data[:int(target.shape[0]*0.6),].shape , count_vectorized_data[int(target.shape[0]*0.6)+1:,].shape

((57510,), (57510, 139175), (38340, 139175))

Some slicing needs to be done, hopefully random, to construct an actual train/validation set. The original test file does not have target values and cannot be used for validation. Next is a fitting to a logit model. The solver 'saga' was used because the default 'liblinear' would fail with the error "'liblinear' does not support multinomial classification". We are inputing into the fitting model only the column that corresponds to 'toxic'. This model basically classifies comments into toxic/non-toxic classes. 

In [61]:
log_reg = LogisticRegression(multi_class='multinomial', penalty ='l1',solver='saga', tol=0.1)
model = log_reg.fit(count_vectorized_data[:int(target.shape[0]*0.6),], target[:int(target.shape[0]*0.6),0])
score = log_reg.score(count_vectorized_data[int(target.shape[0]*0.6)+1:,], target[int(target.shape[0]*0.6)+1:,0])
print("The score for this class is: %.3f" %score) 

The score for this class is: 0.907


In [62]:
train_samples  = int(target.shape[0]*0.6) 

Fitting the model for every class. Not necesarily very smart. This s

In [64]:
scores =[] 
for i in range(target.shape[1]):
    model = log_reg.fit(count_vectorized_data[:train_samples,], target[:train_samples,i])
    score = log_reg.score(count_vectorized_data[train_samples+1:,], target[train_samples+1:,0])
    print("The score for this class is: %.3f" %score)
    scores.append(score)
    
scores

The score for this class is: 0.906
The score for this class is: 0.904
The score for this class is: 0.905
The score for this class is: 0.903
The score for this class is: 0.905
The score for this class is: 0.904


[0.90646844027125717,
 0.90393844548774127,
 0.90477308294209702,
 0.90344287949921753,
 0.90456442357850808,
 0.90354720918101195]